#Ritonavir Binding Site Optimization (Thiozole Ring + Phenyl Ring + Amide Group)

## NOTE: *Each group is specific to Ritonavir*

## Here we Use 'OpenBabel for the Classical Ground State calculation

## **THIOZOLE GROUP**

In [149]:
!obabel -:"ccsc(C)n" -O thiazole.xyz --gen3d


1 molecule converted


In [150]:
# Required Imports
from pyscf import gto, dft
from openfermionpyscf import run_pyscf
from openfermion import MolecularData, jordan_wigner
from openfermion.transforms import get_fermion_operator
from openfermion.linalg import get_sparse_operator
import cirq
import numpy as np
from scipy.optimize import minimize

# Step 1: Create the PySCF molecule
# Read the XYZ file generated by Open Babel
with open('thiazole.xyz', 'r') as f:
    lines = f.readlines()[2:]  # Skip the first two lines (atom count and comment line)

# Convert XYZ to PySCF-compatible geometry string
geometry = '\n'.join(lines)

# Create a PySCF molecule using the XYZ atomic coordinates
mol = gto.M(
    atom=geometry,
    basis='sto-3g',
    charge=0,  # Ensure the charge is neutral
    spin=0     # Singlet state
)

# Print PySCF molecule information
print("Molecule Information from PySCF:")
print("Number of atoms:", mol.natm)
print("Number of electrons:", mol.nelectron)
print("Spin:", mol.spin)
print("Geometry (PySCF format):", mol.atom)

# Perform a DFT calculation to obtain the ground state energy
mf = dft.RKS(mol)
mf.xc = 'b3lyp'  # Use a specific functional
mf.kernel()

# Check if the PySCF calculation was successful
if mf.converged:
    print("PySCF calculation successfully converged.")
else:
    print("PySCF calculation did not converge. Please check the setup.")

# Print the ground state energy from DFT
print("Ground state energy (DFT):", mf.e_tot)


Molecule Information from PySCF:
Number of atoms: 13
Number of electrons: 54
Spin: 0
Geometry (PySCF format): C          1.26185       -0.46847        0.12796

C          2.53902       -0.50818        0.52454

S          3.33590        0.91505        1.09783

C          5.03238        0.53504        0.96015

C          5.56807       -0.12830       -0.27155

N          5.82243        0.89733        1.91512

H          0.76873       -1.36023       -0.24317

H          0.67868        0.44705        0.15432

H          3.08363       -1.44755        0.48584

H          4.78539       -0.45437       -0.95811

H          6.16878       -0.99507        0.02025

H          6.21431        0.57875       -0.79984

H          5.24869        1.36246        2.62825

converged SCF energy = -602.486710641353
PySCF calculation successfully converged.
Ground state energy (DFT): -602.4867106413526


In [151]:
# Step 2: Convert the molecule to an OpenFermion MolecularData object
molecule_name = 'thiazole_molecule'  # Short name for the molecule

# Create MolecularData object
molecular_data = MolecularData(geometry=geometry, basis='sto-3g', charge=0, multiplicity=1, filename=molecule_name)

# Run PySCF calculations and populate molecular data
try:
    molecular_data = run_pyscf(molecular_data)
except Exception as e:
    print(f"Error running PySCF through OpenFermion: {e}")

# Save the MolecularData object to a file
molecular_data.save()

# Try to get the Fermionic Hamiltonian
try:
    fermionic_hamiltonian = molecular_data.get_molecular_hamiltonian()

    # Save the Fermionic Hamiltonian to a file
    with open('fermionic_hamiltonian.txt', 'w') as f:
        f.write(str(fermionic_hamiltonian))

    print("Fermionic Hamiltonian saved to fermionic_hamiltonian.txt.")
except Exception as e:
    print(f"Error obtaining Fermionic Hamiltonian: {e}")

Fermionic Hamiltonian saved to fermionic_hamiltonian.txt.


In [153]:
from openfermion.transforms import jordan_wigner
from openfermion.ops import FermionOperator


# Step 3: Convert the Hamiltonian to qubit Hamiltonian using Jordan-Wigner transformation
qubit_hamiltonian = jordan_wigner(fermionic_hamiltonian)

In [156]:
with open('qubit_hamiltonian.txt', 'w') as file:
    file.write(str(qubit_hamiltonian))

print("Qubit Hamiltonian has been saved to qubit_hamiltonian.txt.")

Qubit Hamiltonian has been saved to qubit_hamiltonian.txt.
